<a href="https://colab.research.google.com/github/barlettab/Desafio_AluraGoogle/blob/main/ChatBot_ApoioEmocional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip install -q -U google-generativeai

In [28]:
# Substitua com sua chave de API do Google
GOOGLE_API_KEY = "SUA_CHAVE"
genai.configure(api_key=GOOGLE_API_KEY)  # Configura a chave da API do Google

In [36]:
# Importa a biblioteca para interagir com a API do Google
import google.generativeai as genai
# Importa o módulo de expressões regulares
import re
import textwrap
from IPython.display import display, Markdown

In [31]:
# Configurações de geração e segurança
generation_config = {
    "candidate_count": 1,
    "temperature": 0.7,  # Temperatura para geração (controla a criatividade)
}

safety_settings = {
    "HARASSMENT": "BLOCK_NONE",
    "HATE": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "DANGEROUS": "BLOCK_NONE",

}

In [32]:
# Inicializando o modelo
model = genai.GenerativeModel(model_name="gemini-1.0-pro", generation_config=generation_config,)

In [33]:
# Palavras sensíveis que acionam respostas específicas
sensitive_words = [
    "automutilação",
    "suicídio",
    "abuso",
    "violência",
    "agressão",
    "estupro",
    "pedofilia",
    "drogas",
    "alcoolismo",
    "vício",
    "transtorno alimentar",
    "depressão",
    "ansiedade",
]

In [34]:
# Dicionário de respostas personalizadas para tópicos sensíveis
sensitive_responses = {
    "automutilação": (
        "Sinto muito em saber que você está se machucando. Você não precisa passar por isso sozinho.\n"
        "Existem pessoas que se preocupam com você e querem ajudar.\n"
        "Ligue para o CVV (Centro de Valorização da Vida) no 188 ou acesse https://cvv.org.br/page/2/ para conversar com alguém.\n"
        "Você também pode encontrar apoio online em https://prevencaoevida.com.br/."
    ),
    "suicídio": (
        "Se você está pensando em suicídio, saiba que existe ajuda disponível. Existem pessoas que querem te ouvir! \n"
        "Ligue para o CVV (Centro de Valorização da Vida) no 188 ou acesse https://cvv.org.br/page/2/ para conversar com alguém.\n"
        "Você também pode encontrar apoio online em https://prevencaoevida.com.br/."
    ),
    "abuso": (
        "Sinto muito em saber que você está passando por isso. Abuso nunca é ok! Saiba que isto não é culpa sua e que você merece, sim, ser tratado com respeito.\n"
        "Existem recursos disponíveis para te ajudar. Ligue para o Disque 180 (violência contra mulher) ou acesse https://www.gov.br/mdh/pt-br/assuntos/violencia-contra-a-mulher/canais-de-atendimento para obter mais informações.\n"
        "Disque 125 para violação dos Direitos da Criança e do Adolescente."
        "Disque 100 para violação dos Direitos Humanos."
    ),
    "violência": (
        "Violência física ou emocional nunca é aceitável. Você merece se sentir seguro(a).\n"
        "Se você ou alguém que conhece está em situação de violência, ligue para o 190 (emergência) ou Disque 180 (violência contra mulher).\n"
        "Você também pode encontrar apoio online em https://www.gov.br/mdh/pt-br/assuntos/denuncie-violencia-contra-a-mulher/violencia-contra-a-mulher"
    ),
    "agressão": (
        "Agressão nunca é ok. Você tem o direito de se sentir seguro(a).\n"
        "Se você ou alguém que conhece está em situação de violência, ligue para o 190 (emergência) ou Disque 180 (violência contra mulher).\n"
        "Você também pode encontrar apoio online em https://www.gov.br/mdh/pt-br/assuntos/denuncie-violencia-contra-a-mulher/violencia-contra-a-mulher"
    ),
    "estupro": (
        "Estupro é um crime hediondo. Eu não consigo imaginar o tamanho da sua dor! Mas você não está sozinho(a).\n"
        "Ligue imediatamente para o 180 (Central de Atendimento à Mulher) ou 190 (emergência) para denunciar e receber apoio.\n"
        "Você também pode encontrar ajuda online em https://www.gov.br/mdh/pt-br/assuntos/violencia-contra-a-mulher."
    ),
    "pedofilia": (
        "Abuso sexual infantil é crime. Se você ou alguém que conhece está passando por isso, NÃO hesite em denunciar.\n"
        "Ligue para o Disque 100 (Criança e Adolescente) ou 181 (Disque Denúncia).\n"
        "Você também pode encontrar informações sobre como denunciar em https://www.gov.br/mdh/pt-br/assuntos/crianca-e-adolescente/meninas-sem-medo."
    ),
    "drogas": (
        "O uso de drogas pode trazer sérios problemas para a saúde. Se você ou alguém que conhece precisa de ajuda, existem recursos disponíveis.\n"
        "Procure um Centro de Atendimento Psicossocial (CAPS) gratuito na sua cidade. CAPS oferecem atendimento especializado para pessoas com dependência química.\n"
        "Você também pode encontrar informações sobre programas de apoio e tratamento em sites do governo, como https://www.gov.br/mds/pt-br/obid/tratamento-e-reinsercao-social."
    ),
    "alcoolismo": (
        "O alcoolismo é uma doença séria, mas tratável. Se você ou alguém que conhece precisa de ajuda, existem recursos disponíveis.\n"
        "Procure um Centro de Atendimento Psicossocial (CAPS) gratuito na sua cidade ou ligue para o 132 (serviço gratuito de assistência em saúde mental).\n"
        "Você também pode encontrar informações sobre clínicas especializadas e grupos de apoio em sites do governo ou ONGs."
        "Acesse o site para mais informações https://www.gov.br/mds/pt-br/obid/tratamento-e-reinsercao-social."
    ),
    "vício": (
        "Existem diferentes tipos de vícios e a abordagem do tratamento varia. Se você ou alguém que conhece precisa de ajuda, existem recursos disponíveis.\n"
        "Procure um Centro de Atendimento Psicossocial (CAPS) gratuito na sua cidade ou converse com um profissional de saúde mental para te orientar sobre o melhor caminho.\n"
        "Você também pode encontrar grupos de apoio online ou presenciais para te dar suporte, acesse o link para mais informações https://www.gov.br/mds/pt-br/obid/tratamento-e-reinsercao-social."
    ),
    "transtorno alimentar": (
        "Transtornos alimentares são doenças sérias que afetam a relação com a comida. Se você ou alguém que conhece precisa de ajuda, existem recursos disponíveis.\n"
        "Procure um Centro de Atendimento Psicossocial (CAPS) gratuito na sua cidade ou converse com um(a) nutricionista ou psicólogo(a) especializado(a) em transtornos alimentares.\n"
        "Você também pode encontrar ONGs e grupos de apoio online ou presenciais para te dar suporte como o Ambulim"
        "Acesse o link para saber mais sobre o Ambulim: https://ambulim.org.br/"
    ),
    "depressão": (
        "A depressão é uma doença comum, mas tratável. Se você acha que pode estar deprimido(a), procure ajuda profissional.\n"
        "Converse com um(a) psicólogo(a) ou médico(a) para receber um diagnóstico e iniciar o tratamento adequado.\n"
        "Você também pode encontrar informações e grupos de apoio online ou presenciais para te dar suporte."
    ),
    "ansiedade": (
        "A ansiedade é uma sensação normal, mas pode se tornar excessiva e atrapalhar a vida cotidiana. Se você sofre de ansiedade, converse com um(a) psicólogo(a) ou médico(a).\n"
        "Existem tratamentos eficazes para a ansiedade, como terapia cognitivo-comportamental e técnicas de relaxamento.\n"
        "Você também pode encontrar grupos de apoio online ou presenciais para te dar suporte."
    )
}


In [ ]:
# Inicia uma sessão de chat com histórico vazio
chat = model.start_chat(history=[])

# Solicita ao usuário que insira seu nome
user_name = input("Olá! Qual é o seu nome? ")
print()


# Apresentação do Chatbot
display(Markdown("\n**🤖 Bem-vindo ao Chatbot de Saúde Mental.**\n"))
display(Markdown("🤖 Estou aqui para ouvir e oferecer suporte.\n"))
display(Markdown("🤖 Sinta-se à vontade para falar sobre o que estiver passando.\n"))
display(Markdown("🤖 Quando quiser encerrar a conversa digite **'fim'**.\n"))



# Loop de conversação
while True:
    # Solicita a entrada do usuário e obtém a resposta do chatbot
    print() ## pula linha
    user_input = input(f"{user_name}: ")
    print() ## pula linha

    # Se o usuário digitar "fim", encerra o chat
    if user_input.lower() == "fim":
        print("🤖 Obrigado por conversar comigo. Espero que tenha te ajudado de alguma forma.\n")
        break

    # Envia a entrada do usuário para o chatbot e obtém a resposta
    response = chat.send_message(user_input)

    # Processa e formata a resposta do chatbot
    response_text = response.text.strip()

    # Verifica palavras sensíveis
    if any(word in user_input.lower() for word in sensitive_words):
      # Se palavras sensíveis forem detectadas, fornece uma resposta sensível
      display(Markdown("\n 🤖 Esse assunto é um tanto delicado... eu posso te oferecer alguns conselhos que podem ser úteis?\n"))
      user_response = input(f"{user_name}: ")
      print()
      if "sim" in user_response.lower():  # Verifica se a entrada do usuário contém "sim"
        # Encontra a palavra sensível na entrada do usuário
        sensitive_word = next((word for word in sensitive_words if word in user_input.lower()), None)
        if sensitive_word in sensitive_responses:
          print(f"🤖 {sensitive_responses[sensitive_word]}")
      else:
          print("🤖 Sinto muito não poder ajudar. Se precisar de mim, estarei aqui 🥰")
    else:
      # Exibe a mensagem do usuário seguida pela resposta do chatbot
      display(Markdown(f"**🤖**: {response_text}"))
      print("-----------------")


Olá! Qual é o seu nome? Daniela




**🤖 Bem-vindo ao Chatbot de Saúde Mental.**


🤖 Estou aqui para ouvir e oferecer suporte.


🤖 Sinta-se à vontade para falar sobre o que estiver passando.


🤖 Quando quiser encerrar a conversa digite **'fim'**.



Daniela: Estou com síndrome do pânico e não consigo me acalmar



**🤖**: **Técnicas de Respiração**

* **Respiração diafragmática:** Coloque uma mão no peito e outra no abdômen. Inspire profundamente pelo nariz, sentindo o abdômen se expandir. Expire lentamente pela boca, contraindo os músculos abdominais.
* **Respiração de caixa:** Inspire por 4 segundos, prenda a respiração por 4 segundos, expire por 4 segundos e prenda a respiração por 4 segundos. Repita por vários minutos.

**Técnicas de Relaxamento**

* **Relaxamento muscular progressivo:** Tense e relaxe sistematicamente diferentes grupos musculares do corpo, começando pelos dedos dos pés e subindo até a cabeça.
* **Atenção plena:** Foque no momento presente, prestando atenção à sua respiração, sensações corporais e pensamentos. Não julgue ou tente controlar, apenas observe.
* **Meditação:** Sente-se ou deite-se em uma posição confortável e concentre-se em uma palavra, frase ou imagem calmante.

**Técnicas Cognitivas**

* **Desafio aos pensamentos negativos:** Identifique os pensamentos catastróficos ou irracionais e questione-os com evidências.
* **Reestruturação cognitiva:** Reformule pensamentos negativos em declarações mais positivas e realistas.
* **Distração:** Envolva-se em atividades que distraiam sua mente dos pensamentos ansiosos, como ler, ouvir música ou assistir a um filme.

**Outras dicas**

* **Exercícios regulares:** A atividade física libera endorfinas, que têm efeitos calmantes.
* **Sono suficiente:** Privação de sono pode piorar a ansiedade.
* **Evite cafeína e álcool:** Essas substâncias podem aumentar a ansiedade.
* **Conecte-se com outras pessoas:** Falar sobre seus sentimentos com pessoas de confiança pode ajudar a reduzir o isolamento.
* **Procure ajuda profissional:** Se você estiver lutando para controlar sua síndrome do pânico por conta própria, considere procurar um terapeuta ou psiquiatra.

Lembre-se de que a síndrome do pânico é uma condição tratável e que você não está sozinho. Com os cuidados e estratégias adequados, você pode aprender a gerenciar seus sintomas e melhorar sua qualidade de vida.

-----------------

